### EDA

- gender 성별
- age 나이
- height 키
- weight 몸무게
- eyesight(left, right) 왼쪽 시력, 오른쪽 시력
- hearing(left, right) 왼쪽 청력, 오른쪽 청력

- systolic 최고혈압(대동맥 내로 보내지는 심실 수축기의 혈압)
- relaxation 최저혈압
- fasting blood sugar 공복혈당
- Cholesterol 총 콜레스테롤
- triglyceride 중성지방
- HDL 고밀도지단백 (콜레스테롤)
- LDL 저밀도지단백 (콜레스테롤)
- hemoglobin 헤모글로빈 수치
- Urine protein 단백뇨
- serum creatinine 혈액 또는 소변에서 크레아티닌 양 측정 - *신장*관련

- AST 아스파르테이트 아미노 전달효소 *간*
- ALT 알라닌 아미노 전달효소 *간* 
    - AST, ALT의 정상 범위 ~40, 경도 40~ 200, 중등도 200~400, 중증 400~
- Gtp 감마 Gtp간 기능 검사 *알코올 관련*
    - 정상 ~50, 가벼운 증가 51~100, 중간정도의 증가 101~200, 고도의 증가 201~500, 초고도 증가 500~
    - 술 마시지 않는 사람도 높게 나올 수 있다.
    
- oral 구강검사 여부
- dental caries 충치 갯수
- tartar 치석존재여부

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [5]:
df = pd.read_csv('./data/smoking.csv')
df.shape  # (55693, 27)

df.head()
type(df)

(55692, 27)

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0


pandas.core.frame.DataFrame

In [6]:
### 비만 데이터프레임 ###

df_obe = df.iloc[:, [2,3,4,5, 10,11,12,13,14,15,16]]
df_obe
rename = df_obe.rename(columns={'height(cm)':'height', 'weight(kg)':'weight', 'waist(cm)':'waist'})
rename

rename.insert(3, 'BMI', rename['weight']/(rename['height']*rename['height']), 0)

target = df.iloc[:,-1]
obesity = pd.concat([rename, target], axis = 1)
obesity

,age,height(cm),weight(kg),waist(cm),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL
0,40,155,60,81.3,114.0,73.0,94.0,215.0,82.0,73.0,126.0
1,40,160,60,81.0,119.0,70.0,130.0,192.0,115.0,42.0,127.0
2,55,170,60,80.0,138.0,86.0,89.0,242.0,182.0,55.0,151.0
3,40,165,70,88.0,100.0,60.0,96.0,322.0,254.0,45.0,226.0
4,40,155,60,86.0,120.0,74.0,80.0,184.0,74.0,62.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...
55687,40,170,65,75.0,110.0,68.0,89.0,213.0,99.0,75.0,118.0
55688,45,160,50,70.0,101.0,62.0,89.0,166.0,69.0,73.0,79.0
55689,55,160,50,68.5,117.0,72.0,88.0,158.0,77.0,79.0,63.0
55690,60,165,60,78.0,133.0,76.0,107.0,210.0,79.0,48.0,146.0


,age,height,weight,waist,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL
0,40,155,60,81.3,114.0,73.0,94.0,215.0,82.0,73.0,126.0
1,40,160,60,81.0,119.0,70.0,130.0,192.0,115.0,42.0,127.0
2,55,170,60,80.0,138.0,86.0,89.0,242.0,182.0,55.0,151.0
3,40,165,70,88.0,100.0,60.0,96.0,322.0,254.0,45.0,226.0
4,40,155,60,86.0,120.0,74.0,80.0,184.0,74.0,62.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...
55687,40,170,65,75.0,110.0,68.0,89.0,213.0,99.0,75.0,118.0
55688,45,160,50,70.0,101.0,62.0,89.0,166.0,69.0,73.0,79.0
55689,55,160,50,68.5,117.0,72.0,88.0,158.0,77.0,79.0,63.0
55690,60,165,60,78.0,133.0,76.0,107.0,210.0,79.0,48.0,146.0


,age,height,weight,BMI,waist,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,smoking
0,40,155,60,0.002497,81.3,114.0,73.0,94.0,215.0,82.0,73.0,126.0,0
1,40,160,60,0.002344,81.0,119.0,70.0,130.0,192.0,115.0,42.0,127.0,0
2,55,170,60,0.002076,80.0,138.0,86.0,89.0,242.0,182.0,55.0,151.0,1
3,40,165,70,0.002571,88.0,100.0,60.0,96.0,322.0,254.0,45.0,226.0,0
4,40,155,60,0.002497,86.0,120.0,74.0,80.0,184.0,74.0,62.0,107.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55687,40,170,65,0.002249,75.0,110.0,68.0,89.0,213.0,99.0,75.0,118.0,0
55688,45,160,50,0.001953,70.0,101.0,62.0,89.0,166.0,69.0,73.0,79.0,0
55689,55,160,50,0.001953,68.5,117.0,72.0,88.0,158.0,77.0,79.0,63.0,0
55690,60,165,60,0.002204,78.0,133.0,76.0,107.0,210.0,79.0,48.0,146.0,0


In [7]:
## 간+심혈관 데이터 프레임#
target = df.loc[:, 'smoking']
target

df_li = df.iloc[:, 10:18]
#df_li

df_bl = df.iloc[:, 20:23]
#df_bl

li_bl = pd.concat([df_li,df_bl],axis=1)
liver_blood = pd.concat([li_bl,target], axis=1)
liver_blood

0        0
1        0
2        1
3        0
4        0
        ..
55687    0
55688    0
55689    0
55690    0
55691    1
Name: smoking, Length: 55692, dtype: int64

,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,AST,ALT,Gtp,smoking
0,114.0,73.0,94.0,215.0,82.0,73.0,126.0,12.9,18.0,19.0,27.0,0
1,119.0,70.0,130.0,192.0,115.0,42.0,127.0,12.7,22.0,19.0,18.0,0
2,138.0,86.0,89.0,242.0,182.0,55.0,151.0,15.8,21.0,16.0,22.0,1
3,100.0,60.0,96.0,322.0,254.0,45.0,226.0,14.7,19.0,26.0,18.0,0
4,120.0,74.0,80.0,184.0,74.0,62.0,107.0,12.5,16.0,14.0,22.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
55687,110.0,68.0,89.0,213.0,99.0,75.0,118.0,12.3,14.0,7.0,10.0,0
55688,101.0,62.0,89.0,166.0,69.0,73.0,79.0,14.0,20.0,12.0,14.0,0
55689,117.0,72.0,88.0,158.0,77.0,79.0,63.0,12.4,17.0,11.0,12.0,0
55690,133.0,76.0,107.0,210.0,79.0,48.0,146.0,14.4,20.0,19.0,18.0,0


In [8]:
#치아 데이터 프레임#
target = df.loc[:, 'smoking']
target
df_or = df.iloc[:, 23:26]
df_or
oral = pd.concat([df_or,target], axis=1)
oral

0        0
1        0
2        1
3        0
4        0
        ..
55687    0
55688    0
55689    0
55690    0
55691    1
Name: smoking, Length: 55692, dtype: int64

NameError: name 'df_or' is not defined